In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
from glob import glob
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns
import matplotlib
from math import pow, sqrt, ceil
from collections import OrderedDict

from sklearn.metrics import accuracy_score, cohen_kappa_score, confusion_matrix

In [3]:
from Analysis.annotations import Annotation
from Analysis.image import Image
from Analysis.expert import Expert, DatasetType, ProjectType

In [4]:
from utils import *

In [5]:
path = Path("Results/Asthma/FirstRound")
files = glob(str(path)+"/Participants/*.txt")
gt_path = str(path)+"/GT/ground truth.txt"

users_first_round = []
ground_truth = Expert("GT", BBType.GroundTruth, DatasetType.Asthma, ProjectType.Annotation)
ground_truth.add_file(gt_path)

for file_name in files:
    participant = "Participant_" + Path(file_name).stem.split("_")[-1]
    
    expert = Expert(participant, BBType.Detected, DatasetType.Asthma, ProjectType.Annotation)
    expert.add_file(file_name)

    users_first_round.append(expert)

In [6]:
ground_truth

GT:  Annos: 458 Seconds: 0.0 Type: Asthma Project: Annotation

In [7]:
for user in users_first_round:
    print(str(user) + " mIoU: {}".format(user.calc_MIoU(ground_truth)))

Participant_11:  Annos: 408 Seconds: 114.5 Type: Asthma Project: Annotation mIoU: 0.765555282730989
Participant_12:  Annos: 410 Seconds: 164.76190476190476 Type: Asthma Project: Annotation mIoU: 0.7422009626832827
Participant_13:  Annos: 402 Seconds: 113.38095238095238 Type: Asthma Project: Annotation mIoU: 0.7753388947263968
Participant_3:  Annos: 397 Seconds: 56.2 Type: Asthma Project: Annotation mIoU: 0.5641239801706175
Participant_4:  Annos: 413 Seconds: 140.25 Type: Asthma Project: Annotation mIoU: 0.7774293463099948
Participant_5:  Annos: 406 Seconds: 96.6 Type: Asthma Project: Annotation mIoU: 0.8226910144113211
Participant_6:  Annos: 402 Seconds: 72.8 Type: Asthma Project: Annotation mIoU: 0.7835845035027322
Participant_7:  Annos: 409 Seconds: 95.8 Type: Asthma Project: Annotation mIoU: 0.5932228978718947
Participant_8:  Annos: 410 Seconds: 85.15 Type: Asthma Project: Annotation mIoU: 0.7490406825045358
Participant_9:  Annos: 406 Seconds: 126.5 Type: Asthma Project: Annotation 

In [8]:
deleted_path = str(path)+"/GT/deleted.txt"
changed_path = str(path)+"/GT/changed.txt"
nms_path = str(path)+"/GT/nms.txt"


deleted = Expert("deleted", BBType.GroundTruth, DatasetType.Asthma, ProjectType.Annotation)
deleted.add_file(deleted_path)

changed = Expert("changed", BBType.GroundTruth, DatasetType.Asthma, ProjectType.Annotation)
changed.add_file(changed_path)

nms = Expert("changed", BBType.GroundTruth, DatasetType.Asthma, ProjectType.Annotation)
nms.add_file(nms_path)


metrics = []
for user in users_first_round:
    total_changed_cells = 0
    found_changed_cells = 0
    for metric in user.calc_metrics(changed):
        total_changed_cells += int(metric['total positives'])
        found_changed_cells += int(metric['total TP'])
        
    total_deleted_cells = 0
    found_deleted_cells = 0
    for metric in user.calc_metrics(deleted):
        total_deleted_cells += int(metric['total positives'])
        found_deleted_cells += int(metric['total TP'])
        
    total_nms_cells = 0
    found_nms_cells = 0
    for metric in user.calc_metrics(nms):
        total_nms_cells += int(metric['total positives'])
        found_nms_cells += int(metric['total TP'])
        
    
    metrics.append([user.expert, total_changed_cells, found_changed_cells, total_deleted_cells, found_deleted_cells, total_nms_cells, found_nms_cells])
pd.DataFrame(metrics, columns=['Name', 'Total Changed', "Found Changed", "Total Deleted", "Found Deleted", "Total NMS", "Found NMS"])

D:\ProgProjekte\Python\Object-Detection-Metrics\lib\Evaluator.py:126: RuntimeWarning: invalid value encountered in true_divide
  rec = acc_TP / npos


,Name,Total Changed,Found Changed,Total Deleted,Found Deleted,Total NMS,Found NMS
0,Participant_11,15,13,5,5,8,4
1,Participant_12,15,13,5,4,8,5
2,Participant_13,15,15,5,3,8,4
3,Participant_3,15,13,5,5,8,6
4,Participant_4,15,14,5,3,8,4
5,Participant_5,15,12,5,5,8,5
6,Participant_6,15,13,5,5,8,4
7,Participant_7,15,11,5,4,8,5
8,Participant_8,15,12,5,3,8,5
9,Participant_9,15,13,5,5,8,3
